## Impport Libraries

In [ ]:
import json
import pandas as pd
from itertools import repeat

## Create f'n to read json data

In [ ]:
def read_json(filename: str) -> dict:
    try:
        with open(filename, "r") as f:
            data = json.loads(f.read())
    except FileNotFoundError as e:
        raise Exception(f"Reading {filename} file encountered an error" + str(e))
    return data

## Create f'n to normalize json data

In [ ]:
def normalize_json(data: dict):
    main_menu_name = ''
    main_menu_item_id = ''
    menu_names_list = []
    menu_values_list = []
    menu_item_ids_list = []
    menu_item_names_list = []
    menu_names_final = []
    menu_levels_list = []
    i = 0
    for key, value in data.items():
        main_menu_name = key
        if isinstance(value, dict):
            for k, v in value.items():
                if k != 'ITEMID':
                    menu_names_list.append(k)
                    menu_values_list.append(v)
                else:
                    main_menu_item_id = v
    x = 0
    df = pd.DataFrame()
    df['menu_name'] = [main_menu_name]
    df['item_id'] = [main_menu_item_id]
    df['menu_level'] = [0]
    df['main_menu_level'] = ['root']

    while x < len(menu_names_list):
        menu_names_final.clear()
        menu_item_names_list.clear()
        menu_item_ids_list.clear()
        menu_levels_list.clear()
        menu_df = prepare_data(menu_names_list, menu_values_list, menu_item_names_list, menu_item_ids_list,
                               menu_names_final, menu_levels_list, x, i, x)
        df = df.append(menu_df)
        x = x + 1
    df.to_excel('output_articlexl.xlsx')

## Create f'n to append levels of json data

In [ ]:
def menu_definition(data: dict, menu_names: list, menu_items: list, menu_levels: list, level) -> list:
    menu_names_list = menu_names
    menu_items_list = menu_items
    manu_levels_list = menu_levels
    menu = []
    menu.append(menu_names)
    menu.append(menu_items)
    menu.append(menu_levels)
    level = level
    if isinstance(data, dict):
        level = level + 1
        for k, v in data.items():
            if k != 'ITEMID':
                menu_names_list.append(k)
                if isinstance(v, dict):
                    menu_definition(v, menu_names_list, menu_items_list, manu_levels_list, level)
                else:
                    print('not a dict')
            else:
                menu_levels.append(level)
                menu_items_list.append(v)
    return menu

## Create f'n to add columns in df 

In [ ]:
def prepare_data(menu_names_list: list, menu_values_list: list, menu_item_names_list: list, menu_item_ids_list: list,
                 menu_names_final: list, menu_levels_list, x, level, main_menu_level):
    menu_names_final.append(menu_names_list[x])
    menu = menu_definition(menu_values_list[x], menu_item_names_list, menu_item_ids_list, menu_levels_list, level)
    menu_names_final.extend(menu[0])
    main_menu_level_list = []
    main_menu_level_list.extend(repeat(main_menu_level, len(menu_names_final)))
    df = pd.DataFrame()
    df['menu_name'] = menu_names_final
    df['item_id'] = menu[1]
    df['menu_level'] = menu[2]
    df['main_menu_level'] = main_menu_level_list
    return df

## Create f'n to provide file name & normalize that data with above f'n

In [ ]:
def main():
    # Read the JSON file as python dictionary
    data = read_json(filename="/content/drive/MyDrive/colab datset/article.json")

    # Normalize the nested python dict
    normalize_json(data=data)

## Assigning variable to our custom module

In [ ]:
if __name__ == '__main__':
    main()